In [5]:
from ase.build import molecule
from ase.io import read, write
import numpy as np

In [8]:
C180 = read("C180-0.xyz")
C60 = read("C60-Ih.xyz")
H20 = molecule("H2O")

write("concat.xyz", C60 + C180 + H20)


In [9]:
!VESTA concat.xyz


(VESTA-gui:30043): Gtk-WARNING **: 14:34:41.589: Negative content width -19 (allocation 1, extents 10x10) while allocating gadget (node button, owner GtkToggleButton)

(VESTA-gui:30043): Gtk-WARNING **: 14:34:41.589: Negative content height -9 (allocation 1, extents 5x5) while allocating gadget (node button, owner GtkToggleButton)


In [10]:
!cat concat.xyz

243
Properties=species:S:1:pos:R:3 C60-Ih=T pbc="F F F"
C        2.16650000       0.59060000       2.58740000
C        3.03780000       0.17660000       1.59180000
C        1.27860000      -0.30980000       3.16790000
C        3.01180000      -1.14340000       1.16540000
C        3.10340000      -1.43350000      -0.19300000
C        3.15030000       1.21060000       0.66820000
C        3.24280000       0.91490000      -0.68590000
C        3.21920000      -0.40230000      -1.12070000
C       -0.43930000       1.35270000       3.12710000
C        0.43630000       2.26180000       2.55420000
C       -0.02960000       0.06330000       3.43790000
C        1.74420000       1.87900000       2.28300000
C        2.35190000       2.26760000       1.09900000
C       -0.26330000       3.02680000       1.63260000
C        0.33740000       3.40540000       0.43730000
C        1.65160000       3.02780000       0.17070000
C       -2.09030000      -0.82250000       2.59550000
C       -2.51110000       

In [44]:
from time import sleep

def force(r_1, r_2, k=1, eq=1):
    extension = r_2 - r_1
    return -k * (extension - eq)

# def force(x_1, x_2, k=1, eq=1):
#     extension = np.linalg.norm(x_1 - x_2)
#     return -k * (extension - eq)

# def energy(x_1, x_2, k):
#     return force(x_1, x_2, k) * np.hypot(x_1, x_2) / 2

def step(v, x_2, F, a_1, dt):
    v = v + F * dt / 2
    x_2 = x_2 * v * dt
    F = force(x_2, a_1)
    v = v + F * dt / 2
    return v, x_2, F, a_1

# def step(v, x_2, F, a_1, dt):
#     global t
#     v = v + F * dt / 2
#     x_2 = x_2 * v * dt
#     F = force(x_2, a_1)
#     v = v + F * dt / 2
#     return v, x_2, F, a_1

n = 2
r = np.zeroes(shape=(n, 3))
r[0][0] = 2
v = np.zeroes(shape=(n, 3))
F = np.zeroes(shape=(n, 3))

# v = 1
# a_1 = np.array([0.00000000, 0.00000000]).reshape(-1, 1)
# x_2 = np.array([0.00000000, 2.000000000]).reshape(-1, 1)
# F = force(a_1, x_2)
t = 0
dt = 1

with open("output.xyz", "w+") as output:
    print("pos:", x_2)
    print("velocity", float(v))
    print("Force", float(F))


    print(2, file=output)
    print(f"time={t}", file=output)
    print(f"O {float(x_2[0])} {float(x_2[1])} 0.0", file=output)
    print(f"O {float(a_1[0])} {float(a_1[1])} 0.0", file=output)

    for _ in range(100):
        print("pos:", x_2)
        print("velocity", float(v))
        print("Force", float(F))
        v, x_2, F, a_1 = step(v, x_2, F, a_1, dt)
        t += dt

        print(2, file=output)
        print(f"time={t}", file=output)
        print(f"O {float(x_2[0])} {float(x_2[1])} 0.0", file=output)
        print(f"O {float(a_1[0])} {float(a_1[1])} 0.0", file=output)
        
    output.close()


pos: [[0.]
 [2.]]
velocity 1.0
Force -1.0
pos: [[0.]
 [2.]]
velocity 1.0
Force -1.0
pos: [[0.]
 [1.]]
velocity 0.5
Force -0.0
pos: [[0. ]
 [0.5]]
velocity 0.75
Force 0.5
pos: [[0. ]
 [0.5]]
velocity 1.25
Force 0.5
pos: [[0.  ]
 [0.75]]
velocity 1.625
Force 0.25
pos: [[0.    ]
 [1.3125]]
velocity 1.59375
Force -0.3125
pos: [[0.        ]
 [1.88671875]]
velocity 0.994140625
Force -0.88671875
pos: [[0.        ]
 [1.03916931]]
velocity 0.5311965942382812
Force -0.0391693115234375
pos: [[0.        ]
 [0.53165143]]
velocity 0.7457862255396321
Force 0.4683485741261393
pos: [[0.       ]
 [0.5209974]]
velocity 1.2194618106900488
Force 0.4790025961746941
pos: [[0.        ]
 [0.76011599]]
velocity 1.5789051128024356
Force 0.23988400805007948
pos: [[0.        ]
 [1.29132086]]
velocity 1.553186686138186
Force -0.29132086137857893
pos: [[0.        ]
 [1.81756802]]
velocity 0.9987422471492792
Force -0.8175680165992347
pos: [[0.        ]
 [1.07228923]]
velocity 0.5538136258184831
Force -0.0722892260623

/tmp/ipykernel_3238/1914850719.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"O {float(x_2[0])} {float(x_2[1])} 0.0", file=output)
/tmp/ipykernel_3238/1914850719.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"O {float(a_1[0])} {float(a_1[1])} 0.0", file=output)
/tmp/ipykernel_3238/1914850719.py:45: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print(f"O {float(x_2[0])} {float(x_2[1])} 0.0", file=output)
/tmp/ipykernel_3238/1914850719.py:

In [45]:
!ovito output.xyz